In [17]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import math
import numpy as np
from tqdm import tqdm,trange
import re
import nltk
from keras.preprocessing.text import text_to_word_sequence


In [35]:
df_data = pd.read_csv("/content/drive/MyDrive/ticaret-yorum.csv", delimiter=',')


In [ ]:
df_data

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
model = AutoModelForTokenClassification.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
tokenizer = AutoTokenizer.from_pretrained("akdeniz27/bert-base-turkish-cased-ner")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

Downloading:   0%|          | 0.00/956 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/373 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/497k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]


```
[{'entity_group': 'ORG',
  'score': 0.80869347,
  'word': 'Altus Hırdavat Yapı',
  'start': 0,
  'end': 19},
 {'entity_group': 'ORG',
  'score': 0.6942621,
  'word': 'Instagram',
  'start': 75,
  'end': 84}]
  ```



In [ ]:
# data = ("Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country. Many people have been killed that day.",
#         {"entities": [(48, 54, 'Category 1'), (77, 81, 'Category 1'), (111, 118, 'Category 2'), (150, 173, 'Category 3')]})

**read back**
```
with open('dict.csv') as csv_file:
    reader = csv.reader(csv_file)
    mydict = dict(reader)
```

In [7]:
def prepareIt(ex):
  ex = ex.replace('"', " ")
  ex = ex.replace(",", " ")
  ex = ex.replace("'", " ")
  ex = ex.replace("...", " ")
  ex = ex.replace("\n", " ")
  ex = ex.replace("\\n", " ")
  ex = ex.replace("\\t", " ")
  ex = ex.replace("\t", " ")
  ex = ex.replace("\\", " ")
  ex = ex.replace("  ", " ")
  return ex

In [9]:
def splitWords(ex):
  words = text_to_word_sequence(ex)
  return words

In [10]:
def getEntities(tokens):
  tup =()
  entities = []
  for t in tokens:
    start= t["start"]
    end = t["end"]
    ent = t["entity_group"]
    tup = (start, end, ent)
    entities.append(tup)
  return entities

In [11]:
def labelData(data):
  tag=[]
  pre = 0
  for i in data[1]["entities"]:
    a = data[0][i[0]:i[1]].split()
    t = pre + i[1]
    #print(pre, i[0])
    b = data[0][pre:i[0]].split()
    for j in b:
      # print("'%s': '%s'," % (j, "O"))
      tag.append("O")
      pre = i[1]
      for j in range(len(a)): 
        if j == 0:
          # print("'%s': '%s-%s'," % (a[j], "B", i[2]))
           tag.append("B-"+i[2])
        else: 
         # print("'%s': '%s-%s'," % (a[j], "I", i[2]))
         tag.append("I-"+i[2])
  #print(i[1], las)
  las = len(data[0])
  c = data[0][i[1]:las].split()
  for j in range(len(c)):
    if j == len(c) - 1:
      # print("'%s': '%s'" % (c[j], "O"))
      tag.append("O")
    else:
      # print("'%s': '%s'," % (c[j], "O"))
      tag.append("O")

  return tag

In [ ]:
n = 10
a = 'O'
l = [a]*i
l

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [33]:
df = df_data

In [37]:
# take random 2000 data
from sklearn.utils import shuffle
df_data = shuffle(df_data)
df_data
df_data = df_data.sample(n=5000)
df_data

,category,text
46206,bilgisayar,"PlayStation Türkiye Kutudan Oyun Çıkmadı!,Tekn..."
81148,elektronik,LG 4K UHD Televizyonun Ekranında Yatay Çizgi B...
123679,etkinlik-ve-organizasyon,"Cinetech Sinemaları Filmler Gelmiyor,""Kütahya'..."
404641,turizm,"Jolly Tur İptal Talebim Reddediliyor,""Temmuz a..."
284376,mekan-ve-eglence,"Domino's Pizza Aldığım Sos Dökülmüyor,""Son 2 s..."
...,...,...
417425,ulasim,Sunexpress Havayolları Uçuşum İptal Edildi Sun...
418165,ulasim,"Isparta Petrol Turizm Maskesiz Yolcu Taşıma,04..."
277640,medya,"Atv 2013'ten Beri Yarışma İçin Aranmadım.,""Bil..."
150755,gida,7days Croissant Müşteri Hizmetleri İlgilenmiyo...


In [ ]:
from tqdm.auto import tqdm
label_list = []
no_label_data = []
no_label = 1
c = 0
for d in tqdm(df_data["text"]):
  c +=1
  d = prepareIt(d)
  tokens = ner(d)
  d = d.lower()

  words = splitWords(d)

  entities = getEntities(tokens)

  m = (d,{"entities":entities})
  if not m[1]["entities"]:
    print("Error no labeled data ", no_label)
    n = len(words) 
    a = 'O'
    tag= [a]*n
    a = {
      "sentence#": c,
      "words": words,
      "tag": tag
    }
    no_label_data.append(a)
    no_label += 1
  else: 
    tag = labelData(m)
  a = {
      "sentence#": c,
      "words": words,
      "tag": tag
    }
  label_list.append(a)
  

In [ ]:
for l in label_list:
  print(l)

In [53]:
import csv
 
# opening the csv file in 'w' mode
file = open('g4g.csv', 'w', newline ='')
 
with file:
    # identifying header 
    header = ['sentence#', 'words', 'tag']
    writer = csv.DictWriter(file, fieldnames = header)
     
    # writing data row-wise into the csv file
    writer.writeheader()
    for l in label_list:
      sentence_id = l["sentence#"]
      words = tuple(l["words"])
      tag = tuple(l["tag"])
      writer.writerow({'sentence#' : sentence_id,
                     'words': words,
                     'tag': tag})




In [55]:
import pandas as pd

df = pd.read_csv('g4g.csv')

print(df) 

      sentence#                                              words  \
0             1  ('playstation', 'türkiye', 'kutudan', 'oyun', ...   
1             2  ('lg', '4k', 'uhd', 'televizyonun', 'ekranında...   
2             3  ('cinetech', 'sinemaları', 'filmler', 'gelmiyo...   
3             4  ('jolly', 'tur', 'i̇ptal', 'talebim', 'reddedi...   
4             5  ('domino', 's', 'pizza', 'aldığım', 'sos', 'dö...   
...         ...                                                ...   
4995       4996  ('sunexpress', 'havayolları', 'uçuşum', 'i̇pta...   
4996       4997  ('isparta', 'petrol', 'turizm', 'maskesiz', 'y...   
4997       4998  ('atv', '2013', 'ten', 'beri', 'yarışma', 'i̇ç...   
4998       4999  ('7days', 'croissant', 'müşteri', 'hizmetleri'...   
4999       5000  ('baymak', 'yetkili', 'servis', 'klimamı', 'ta...   

                                                    tag  
0     ('O', 'B-ORGANIZATION', 'O', 'B-ORGANIZATION',...  
1     ('O', 'B-PRODUCT', 'I-PRODUCT', 'O', 

```
@software{stefan_schweter_2020_3770924,
  author       = {Stefan Schweter},
  title        = {BERTurk - BERT models for Turkish},
  month        = apr,
  year         = 2020,
  publisher    = {Zenodo},
  version      = {1.0.0},
  doi          = {10.5281/zenodo.3770924},
  url          = {https://doi.org/10.5281/zenodo.3770924}
}
```

```
@software{Taner Akdeniz,
  author       = {Taner Akdeniz},
  title        = {akdeniz27/bert-base-turkish-cased-ner},
  month        = sep,
  year         = 2021,
  url          = {https://huggingface.co/akdeniz27/bert-base-turkish-cased-ner}
}
```
[NER MODEL](https://huggingface.co/busecarik/berturk-sunlp-ner-turkish)